# Vehicle Detection & Confirmation System
## Version 2
Summer 2023 

Created by Vikram Anantha \
Continued from Ben Dwyer's code \
July 27 2023

## Summary

(Same as V1) \
This code is meant to be implemented into Road-Side Systems (RSSs), like traffic cameras, such that it can communicate with vehicles, especially Autonomous Vehicles (AVs).

### Background
(Same as V1) \
One problem that might arise when AVs communicate with RSSs is that a hacker with malicious intent can join the same channel and communicate with the RSSs as if they were the vehicle. To combat this, the RSS can command the vehicle to confirm its identity by performing a specific task. Examples of this include:
 + Displaying a specific pattern on a screen, like a QR code
 + Flashing headlights in a specific pattern
 + Making a sound in a specific pattern

Once the vehicle performs the task, the RSS can confirm this has been done visually, thus confirming the identiy of the vehicle. This visual confirmation, a form of Two Factor Authentication (2FA) is the premise of this code.



### Changes from V1

These were the shortcomings from Version 1:
 + When using live feed, there is about a second of delay per frame to detect if a vehicle is present, and to recognize there is no bounding box. It is much more time to recognize the array in the bounding box, which needs to be done for each vehicle.
 + The code runs a for loop to go through each vehicle and detect the pattern, meaning it verifies each vehicle one by one. Because each vehicle takes a long time to be verified, when multiple vehicles are present, the system will take a _very_ long time.
 + When using a webcam, or a dedicated camera for the computer (as in not a smartphone camera, which uses HDR to make screens appear normal with everything else), the Arduino display is super bright, and cannot be recognized.
 + The last point also doesn't really matter as much, since in the real environment, having a screen on the windshield won't be implemented. Instead, it would most likely leverage the headlights, having them flash a pattern across time, rather than display a pattern across space.

Version 2 solves these problems in these ways:
 + (nothing yet)

Todo (Jul 27 2023):
 + implement Zed camera based object detection
 + run a model to use headlights as the pattern recognition
    + first get that to run on a given video
    + then write arduino code to make the display act like headlights
    + then get it to work in real time
    + then see if the pattern recognition is needed with ML or just if statements
 + try to get multiple vehicles to work fast (idk how yet)
    + zed api?
 + Priority 1: documentation

Meeting with Daijang Suo [Jul 24 2023]


ZED obj detection should be faster

https://www.stereolabs.com/docs/object-detection/


Detect vehicle + detect headlights + detect headlight pattern

^^ simultaneously



https://arxiv.org/pdf/1906.03683.pdf

Action-State Joint Learning-Based Vehicle Taillight

Recognition in Diverse Actual Traffic Scenes



Document all these steps rigorouly



https://github.com/TechToker/CarLightSignalsDetection

^^ sliding window of frames?


Api provided by zed camera to do categorization


Look through https://www.stereolabs.com/docs/object-detection/custom-od/

To get obj detection and custom detection




Todo

Make the headlights work over time on the display

Get multiple vehicle detection to work well

And also get multiple vehicles to work fast (with zed api)

Documentation




In [2]:
import glob
from roboflow import Roboflow
from ultralytics import YOLO
import time
import cv2
import numpy as np
from datetime import datetime

import os

avg = lambda x: sum(x)/len(x)

## Zed Camera Code: Object Detection Birds Eye

In [2]:
########################################################################
#
# Copyright (c) 2022, STEREOLABS.
#
# All rights reserved.
#
# THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS
# "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT
# LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR
# A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT
# OWNER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL,
# SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT
# LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE,
# DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY
# THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT
# (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE
# OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.
#
########################################################################

"""
    This sample demonstrates how to capture 3D point cloud and detected objects
    with the ZED SDK and display the result in an OpenGL window.
"""

import sys
import numpy as np
import cv2
import pyzed.sl as sl
# import helpercode.ogl_viewer.viewer as gl
import helpercode.cv_viewer.tracking_viewer as cv_viewer
from helpercode.batch_system_handler import *


##
# Variable to enable/disable the batch option in Object Detection module
# Batching system allows to reconstruct trajectories from the object detection module by adding Re-Identification / Appareance matching.
# For example, if an object is not seen during some time, it can be re-ID to a previous ID if the matching score is high enough
# Use with caution if image retention is activated (See batch_system_handler.py) :
#   --> Images will only appear if an object is detected since the batching system is based on OD detection.
USE_BATCHING = False

# if __name__ == "__main__":
if True:
    print("Running object detection ... Press 'Esc' to quit")
    zed = sl.Camera()
    
    # Create a InitParameters object and set configuration parameters
    init_params = sl.InitParameters()
    init_params.coordinate_units = sl.UNIT.METER
    init_params.coordinate_system = sl.COORDINATE_SYSTEM.RIGHT_HANDED_Y_UP  
    init_params.depth_mode = sl.DEPTH_MODE.ULTRA
    init_params.depth_maximum_distance = 20
    is_playback = False                             # Defines if an SVO is used
        
    # If applicable, use the SVO given as parameter
    # Otherwise use ZED live stream
    if len(sys.argv) == 2:
        filepath = sys.argv[1]
        print("Using SVO file: {0}".format(filepath))
        init_params.svo_real_time_mode = True
        init_params.set_from_svo_file(filepath)
        is_playback = True

    status = zed.open(init_params)
    if status != sl.ERROR_CODE.SUCCESS:
        print(repr(status))
        exit()


    # Enable positional tracking module
    positional_tracking_parameters = sl.PositionalTrackingParameters()
    # If the camera is static in space, enabling this setting below provides better depth quality and faster computation
    positional_tracking_parameters.set_as_static = True
    zed.enable_positional_tracking(positional_tracking_parameters)

    # Enable object detection module
    batch_parameters = sl.BatchParameters()
    if USE_BATCHING:
        batch_parameters.enable = True
        batch_parameters.latency = 2.0
        batch_handler = BatchSystemHandler(batch_parameters.latency*2)
    else:
        batch_parameters.enable = False
    obj_param = sl.ObjectDetectionParameters(batch_trajectories_parameters=batch_parameters)
        
    obj_param.detection_model = sl.OBJECT_DETECTION_MODEL.MULTI_CLASS_BOX_FAST
    # Defines if the object detection will track objects across images flow.
    obj_param.enable_tracking = True
    zed.enable_object_detection(obj_param)

    camera_infos = zed.get_camera_information()
    # Create OpenGL viewer
    # viewer = gl.GLViewer()
    point_cloud_res = sl.Resolution(min(camera_infos.camera_configuration.resolution.width, 720), min(camera_infos.camera_configuration.resolution.height, 404)) 
    point_cloud_render = sl.Mat()
    # viewer.init(camera_infos.camera_model, point_cloud_res, obj_param.enable_tracking)
    
    # Configure object detection runtime parameters
    obj_runtime_param = sl.ObjectDetectionRuntimeParameters()
    detection_confidence = 60
    obj_runtime_param.detection_confidence_threshold = detection_confidence
    # To select a set of specific object classes
    obj_runtime_param.object_class_filter = [sl.OBJECT_CLASS.PERSON]
    # To set a specific threshold
    obj_runtime_param.object_class_detection_confidence_threshold = {sl.OBJECT_CLASS.PERSON: detection_confidence} 

    # Runtime parameters
    runtime_params = sl.RuntimeParameters()
    runtime_params.confidence_threshold = 50

    # Create objects that will store SDK outputs
    point_cloud = sl.Mat(point_cloud_res.width, point_cloud_res.height, sl.MAT_TYPE.F32_C4, sl.MEM.CPU)
    objects = sl.Objects()
    image_left = sl.Mat()

    # Utilities for 2D display
    display_resolution = sl.Resolution(min(camera_infos.camera_configuration.resolution.width, 1280), min(camera_infos.camera_configuration.resolution.height, 720))
    image_scale = [display_resolution.width / camera_infos.camera_configuration.resolution.width
                 , display_resolution.height / camera_infos.camera_configuration.resolution.height]
    image_left_ocv = np.full((display_resolution.height, display_resolution.width, 4), [245, 239, 239,255], np.uint8)

    # Utilities for tracks view
    camera_config = zed.get_camera_information().camera_configuration
    tracks_resolution = sl.Resolution(400, display_resolution.height)
    track_view_generator = cv_viewer.TrackingViewer(tracks_resolution, camera_config.fps, init_params.depth_maximum_distance)
    track_view_generator.set_camera_calibration(camera_config.calibration_parameters)
    image_track_ocv = np.zeros((tracks_resolution.height, tracks_resolution.width, 4), np.uint8)

    # Will store the 2D image and tracklet views 
    global_image = np.full((display_resolution.height, display_resolution.width+tracks_resolution.width, 4), [245, 239, 239,255], np.uint8)

    # Camera pose
    cam_w_pose = sl.Pose()
    cam_c_pose = sl.Pose()

    quit_app = False

    # while(viewer.is_available() and (quit_app == False)):
    while((quit_app == False)):
        if zed.grab(runtime_params) == sl.ERROR_CODE.SUCCESS:
            # Retrieve objects
            returned_state = zed.retrieve_objects(objects, obj_runtime_param)
            
            if (returned_state == sl.ERROR_CODE.SUCCESS and objects.is_new):
                # Retrieve point cloud
                zed.retrieve_measure(point_cloud, sl.MEASURE.XYZRGBA,sl.MEM.CPU, point_cloud_res)
                point_cloud.copy_to(point_cloud_render)
                # Retrieve image
                zed.retrieve_image(image_left, sl.VIEW.LEFT, sl.MEM.CPU, display_resolution)
                image_render_left = image_left.get_data()
                # Get camera pose
                zed.get_position(cam_w_pose, sl.REFERENCE_FRAME.WORLD)

                update_render_view = True
                # update_3d_view = True
                update_tracking_view = True

                if USE_BATCHING:
                    zed.get_position(cam_c_pose, sl.REFERENCE_FRAME.CAMERA)
                    objects_batch = []
                    zed.get_objects_batch(objects_batch)
                    batch_handler.push(cam_c_pose,cam_w_pose,image_left,point_cloud,objects_batch)
                    cam_c_pose, cam_w_pose, image_left, point_cloud_render, objects = batch_handler.pop(cam_c_pose,cam_w_pose,image_left,point_cloud,objects)
                    
                    image_render_left = image_left.get_data()
                    
                    update_tracking_view = objects.is_new

                    if WITH_IMAGE_RETENTION:
                        update_render_view = objects.is_new
                        # update_3d_view = objects.is_new
                    else:
                        update_render_view = True
                        # update_3d_view = True

                # 3D rendering
                # if update_3d_view:
                    # viewer.updateData(point_cloud_render, objects)

                # 2D rendering
                if update_render_view:
                    np.copyto(image_left_ocv,image_render_left)
                    cv_viewer.render_2D(image_left_ocv,image_scale,objects, obj_param.enable_tracking)
                    global_image = cv2.hconcat([image_left_ocv,image_track_ocv])

                # Tracking view
                if update_tracking_view:
                    track_view_generator.generate_view(objects, cam_w_pose, image_track_ocv, objects.is_tracked)
                    
            cv2.imshow("ZED | 2D View and Birds View",global_image)
            cv2.waitKey(10)

        if (is_playback and (zed.get_svo_position() == zed.get_svo_number_of_frames()-1)):
            print("End of SVO")
            quit_app = True


    cv2.destroyAllWindows()
    # viewer.exit()
    image_left.free(sl.MEM.CPU)
    point_cloud.free(sl.MEM.CPU)
    point_cloud_render.free(sl.MEM.CPU)

    if USE_BATCHING:
        batch_handler.clear()

    # Disable modules and close camera
    zed.disable_object_detection()
    zed.disable_positional_tracking()

    zed.close()

Running object detection ... Press 'Esc' to quit


KeyboardInterrupt: 

## About RoboFlow and Yolo

[Roboflow](https://roboflow.com/) is a platform which houses a vast variety of models, mainly for computer vision. It is often used because it is easy to train the models, and prediction happens over the cloud. For computer vision, Roboflow uses [YOLOv8](https://blog.roboflow.com/whats-new-in-yolov8/) (You Only Look Once), the fastest and most accurate architecture.

Roboflow has a very intuitive process to upload training images and annotate them. For models that detect (rather than just classify), the model not only must be given the image, but also the coordinates on the image of where the object in question is, and what the object is called. Roboflow allows you to create the training data fairly easily, and can then be downloaded to be trained on device.

In Version 1, the training data and the models was stored on RoboFlow, meaning anytime the code required a prediction from the model, it would make a call through the internet. This is ideal for most cases, (i.e. if the user is using a Windows laptop from 1997), however this code is meant to be run on an Edge computer, meaning it should have the capabilities to run the model on device. In this version, the model is trained and run on device.

To do this, the dataset created in RoboFlow is downloaded onto the device, and a YoloV8 model is trained with this dataset.

When the dataset is downloaded, it is stored in a folder (for example, `RC-Vehicle-Detection_v2-1`), which has a couple files
 - `test`
    - `images`
        - folder of the unedited images that were uploaded
    - `labels`
        - txt files with the coords of where the object is and its classification
 - `train`
    - same subfolders as `test`
 - `valid`
    - same subfolders as `test`
 - `data.yaml`
    - the main datafile, has info about where the images are, what the clssifications are, etc. This file is the only file inputted when training the model 
 - `README`s

The model is then trained with this data, and saved.

In [ ]:
# !pip install roboflow


rf = Roboflow(api_key="2Vs9PCO5LGDCSkI0huRq")
project = rf.workspace("meng-thesis-5fidi").project("rc-vehicle-detection_v2")
dataset = project.version(1).download("yolov8")


In [8]:
## Training a custom detector

# based on this tutorial: https://learnopencv.com/train-yolov8-on-custom-dataset/


 
# Load the model.
model = YOLO('yolov8n.pt')
 
# # Training.
results = model.train(
   data=f'{dataset.location}/data.yaml',
   imgsz=640,
   epochs=100,
   batch=8,
   name='yolov8n_rc_v1'
)

# For epochs 100, imgsize 640, it took 769 mins to train

Ultralytics YOLOv8.0.143 🚀 Python-3.8.10 torch-2.0.0 CPU (ARMv8 Processor rev 1 (v8l))
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/home/auto-id/Vikram Anantha Summer 2023/Codes/RC-Vehicle-Detection_v2-1/data.yaml, epochs=100, patience=50, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolov8n_rc_v1, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=t

In [2]:
model = YOLO('runs/detect/yolov8n_rc_v13/weights/best.pt')
# metrics = model.val()

In [10]:
metrics.box.map

0.7549277469889375

In [10]:
results = model(glob.glob('RC-Vehicle-Detection_v2-1/test/images/*.jpg'),
                save=True,
                save_txt=True,
                save_conf=True,
                save_crop=True,
                )
# Takes 3.6 seconds to predict on 35 images, so about 0.1s per image.






    WARNING ⚠️ stream/video/webcam/dir predict source will accumulate results in RAM unless `stream=True` is passed,
    causing potential out-of-memory errors for large sources or long-running streams/videos.

    Usage:
        results = model(source=..., stream=True)  # generator of Results objects
        for r in results:
            boxes = r.boxes  # Boxes object for bbox outputs
            masks = r.masks  # Masks object for segment masks outputs
            probs = r.probs  # Class probabilities for classification outputs

video 1/1 (1/453) /home/auto-id/Vikram Anantha Summer 2023/Codes/Videos/drive_05/drive_05_vikram2023.mp4: 384x640 (no detections), 353.8ms
video 1/1 (2/453) /home/auto-id/Vikram Anantha Summer 2023/Codes/Videos/drive_05/drive_05_vikram2023.mp4: 384x640 1 RC-Car, 223.9ms
video 1/1 (3/453) /home/auto-id/Vikram Anantha Summer 2023/Codes/Videos/drive_05/drive_05_vikram2023.mp4: 384x640 (no detections), 318.1ms
video 1/1 (4/453) /home/auto-id/Vikram Anantha Su

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 keys: ['boxes']
 masks: None
 names: {0: 'RC-Car'}
 orig_img: array([[[ 55,  60,  46],
         [ 55,  60,  46],
         [ 55,  60,  46],
         ...,
         [101, 104, 105],
         [101, 104, 105],
         [101, 104, 105]],
 
        [[ 55,  60,  46],
         [ 55,  60,  46],
         [ 55,  60,  46],
         ...,
         [101, 104, 105],
         [101, 104, 105],
         [101, 104, 105]],
 
        [[ 55,  60,  46],
         [ 55,  60,  46],
         [ 55,  60,  46],
         ...,
         [101, 104, 105],
         [101, 104, 105],
         [101, 104, 105]],
 
        ...,
 
        [[154, 180, 193],
         [164, 189, 202],
         [124, 150, 162],
         ...,
         [122, 125, 122],
         [113, 117, 113],
         [134, 138, 134]],
 
        [[175, 201, 214],
         [150, 175, 188],
         [122, 147, 160],
         ...,
         [12

In [9]:
# Score for the Model
realdata_path = 'RC-Vehicle-Detection_v2-1/test/labels/*.txt'
fakedata_path = 'runs/detect/predict2/labels/*.txt'

realdata_files = glob.glob(realdata_path)
fakedata_files = glob.glob(fakedata_path)
if (len(realdata_files) != len(fakedata_files)):
    raise "WrongFilePathException"
confs = []
bounds_diffs = []
for i in range(len(realdata_files)):
    with open(realdata_files[i], 'r') as rfile:
        robj, rxl, rxr, ryl, ryr = map(float, rfile.readline().split(" "))
    with open(fakedata_files[i], 'r') as ffile:
        fobj, fxl, fxr, fyl, fyr, fconf = map(float, ffile.readline().split(" "))
        
    confs.append(fconf)
    bounds_diffs.append(abs(rxl-fxl)*100)
    bounds_diffs.append(abs(rxr-fxr)*100)
    bounds_diffs.append(abs(ryl-fyl)*100)
    bounds_diffs.append(abs(ryr-fyr)*100)
    
print("Average Confidence:             ", avg(confs))
print("Average Bounding Box Difference:", avg(bounds_diffs), "% of the image")

# Average Confidence:              0.8520260857142857
# Average Bounding Box Difference: 0.6273896666666672% of the image

Average Confidence:              0.8520260857142857
Average Bounding Box Difference: 0.6273896666666672 % of the image


In [11]:
model("Videos/drive_05/drive_05_vikram2023.mp4",
      save=True,
      save_txt=True,
      save_conf=True,
      save_crop=True
      )

# 15.5s video, with save,save_txt,save_conf,save_crop:
# 160 seconds to predict
# 1/3 seconds to predict per frame



    WARNING ⚠️ stream/video/webcam/dir predict source will accumulate results in RAM unless `stream=True` is passed,
    causing potential out-of-memory errors for large sources or long-running streams/videos.

    Usage:
        results = model(source=..., stream=True)  # generator of Results objects
        for r in results:
            boxes = r.boxes  # Boxes object for bbox outputs
            masks = r.masks  # Masks object for segment masks outputs
            probs = r.probs  # Class probabilities for classification outputs

video 1/1 (1/453) /home/auto-id/Vikram Anantha Summer 2023/Codes/Videos/drive_05/drive_05_vikram2023.mp4: 384x640 (no detections), 353.3ms
video 1/1 (2/453) /home/auto-id/Vikram Anantha Summer 2023/Codes/Videos/drive_05/drive_05_vikram2023.mp4: 384x640 1 RC-Car, 270.3ms
video 1/1 (3/453) /home/auto-id/Vikram Anantha Summer 2023/Codes/Videos/drive_05/drive_05_vikram2023.mp4: 384x640 (no detections), 258.3ms
video 1/1 (4/453) /home/auto-id/Vikram Anantha Su

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 keys: ['boxes']
 masks: None
 names: {0: 'RC-Car'}
 orig_img: array([[[ 55,  60,  46],
         [ 55,  60,  46],
         [ 55,  60,  46],
         ...,
         [101, 104, 105],
         [101, 104, 105],
         [101, 104, 105]],
 
        [[ 55,  60,  46],
         [ 55,  60,  46],
         [ 55,  60,  46],
         ...,
         [101, 104, 105],
         [101, 104, 105],
         [101, 104, 105]],
 
        [[ 55,  60,  46],
         [ 55,  60,  46],
         [ 55,  60,  46],
         ...,
         [101, 104, 105],
         [101, 104, 105],
         [101, 104, 105]],
 
        ...,
 
        [[154, 180, 193],
         [164, 189, 202],
         [124, 150, 162],
         ...,
         [122, 125, 122],
         [113, 117, 113],
         [134, 138, 134]],
 
        [[175, 201, 214],
         [150, 175, 188],
         [122, 147, 160],
         ...,
         [12

In [7]:
def cv2_livefeed_rfdevice():
  
    # define a video capture object
    vid = cv2.VideoCapture(-1) # use the camera
    vid.set(cv2.CAP_PROP_FRAME_WIDTH, 3840) 
    vid.set(cv2.CAP_PROP_FRAME_HEIGHT, 2160)
    # 4k dimensions
    
    foldername = 'runs/detect/predict5/'
    filename = foldername + 'image0.jpg'
    latencies = []
    count = 0
    while(True):

        # Capture the video frame by frame
        
        for _ in range(4): 
            vid.grab()
        # for some reason cv2 takes the next 5 frames and buffers them
        # this means that whats displayed is 5 frames behind
        # but when each frame takes a second to process, that's a 5 second delay
        # this makes sure its only looking at the current frame
        
        
        ret, frame = vid.read()
        startbig = time.time()
        # Zed cameras have two cameras so only use one of them
        frame = np.hsplit(frame, 2)[0]
        
        
        # do vehicle detection
        model(frame, save=True)
        # opencv_boxed_image = np.array(opencv_boxed_image)[:, :, ::-1].copy() # turns the PIL image to a CV2-ready image
        
        # display frame
        frame = cv2.imread(filename)
        endbig = time.time()
        cv2.imshow('frame', frame)
        
        print(f"{round(endbig-startbig, 2)} seconds overall")
        latencies.append(endbig-startbig)
        # the 'q' button is set as the
        # quitting button you may use any
        # desired button of your choice
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        # current_time = datetime.now().strftime("%H_%M_%S_%f")
        # cv2.imwrite(foldername + 'video_frames/frame_%s.jpg' % current_time, frame)

    # After the loop release the cap object
    vid.release()
    # Destroy all the windows
    cv2.destroyAllWindows()
    
    print("Average Latency: ", round(avg(latencies), 3), "seconds")
    
cv2_livefeed_rfdevice()
# ~0.52 seconds per loop
# ~0.42 seconds of latency


0: 384x640 1 RC-Car, 395.6ms
Speed: 9.8ms preprocess, 395.6ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict5



0.47 seconds overall


0: 384x640 (no detections), 490.1ms
Speed: 5.0ms preprocess, 490.1ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict5



0.56 seconds overall


0: 384x640 (no detections), 514.5ms
Speed: 5.0ms preprocess, 514.5ms inference, 2.7ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict5



...

0.47 seconds overall


0: 384x640 1 RC-Car, 323.5ms
Speed: 5.1ms preprocess, 323.5ms inference, 3.7ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict5



0.41 seconds overall


0: 384x640 1 RC-Car, 323.8ms
Speed: 4.7ms preprocess, 323.8ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict5


0.41 seconds overall
Average Latency:  0.551 seconds


In [3]:
def export_video(input_val, output_val, fps=25):
    # stich images together into video
    # os.system(f"ffmpeg -r {fps} -s 3840x2160 -pattern_type glob -i {input_val} -vcodec libx264 -crf {fps}  -pix_fmt yuv420p {output_val}")
    # !ffmpeg -r 25 -s 1920x1080 -i '/Users/bendwyer/Documents/Academic/2022-2023/Thesis/Vehicle Detection/%04d.png' -vcodec libx264 -crf 25  -pix_fmt yuv420p '/Users/bendwyer/Documents/Academic/2022-2023/Thesis/Vehicle Detection/test.mp4'
    os.system(f"ffmpeg -framerate {fps} -pattern_type glob -i '{input_val}' -c:v libx264 -r 30 -pix_fmt yuv420p {output_val}")
export_video(
    input_val='runs/detect/predict5/video_frames/frame_*.jpg',
    fps=4,
    output_val='runs/detect/predict5/in_the_sun.mp4'
)

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/aarch64-linux-gnu --incdir=/usr/include/aarch64-linux-gnu --arch=arm64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis -

In [2]:
#!/usr/bin/env python3

import sys
import numpy as np

import argparse
import torch
import cv2
import pyzed.sl as sl
from ultralytics import YOLO

from threading import Lock, Thread
from time import sleep
import time

import glob
from roboflow import Roboflow
from ultralytics import YOLO
import time
import cv2
import numpy as np

avg = lambda x: sum(x)/len(x)

# import helpercode.custom.ogl_viewer.viewer as gl
import helpercode.cv_viewer.tracking_viewer as cv_viewer
# from helpercode.batch_system_handler import *

lock = Lock()
run_signal = False
exit_signal = False


def xywh2abcd(xywh, im_shape):
    output = np.zeros((4, 2))

    # Center / Width / Height -> BBox corners coordinates
    x_min = (xywh[0] - 0.5*xywh[2]) #* im_shape[1]
    x_max = (xywh[0] + 0.5*xywh[2]) #* im_shape[1]
    y_min = (xywh[1] - 0.5*xywh[3]) #* im_shape[0]
    y_max = (xywh[1] + 0.5*xywh[3]) #* im_shape[0]

    # A ------ B
    # | Object |
    # D ------ C

    output[0][0] = x_min
    output[0][1] = y_min

    output[1][0] = x_max
    output[1][1] = y_min

    output[2][0] = x_min
    output[2][1] = y_max

    output[3][0] = x_max
    output[3][1] = y_max
    return output

def detections_to_custom_box(detections, im0):
    output = []
    for i, det in enumerate(detections):
        xywh = det.xywh[0]

        # Creating ingestable objects for the ZED SDK
        obj = sl.CustomBoxObjectData()
        obj.bounding_box_2d = xywh2abcd(xywh, im0.shape)
        obj.label = det.cls
        obj.probability = det.conf
        obj.is_grounded = False
        output.append(obj)
    return output


def torch_thread(weights, img_size, conf_thres=0.2, iou_thres=0.45):
    global image_net, exit_signal, run_signal, detections

    print("Intializing Network...")

    model = YOLO(weights)

    while not exit_signal:
        if run_signal:
            lock.acquire()

            img = cv2.cvtColor(image_net, cv2.COLOR_BGRA2RGB)
            # https://docs.ultralytics.com/modes/predict/#video-suffixes
            det = model.predict(img, save=False, imgsz=img_size, conf=conf_thres, iou=iou_thres)[0].cpu().numpy().boxes

            # ZED CustomBox format (with inverse letterboxing tf applied)
            detections = detections_to_custom_box(det, image_net)
            lock.release()
            run_signal = False
        sleep(0.01)


def main():
    global image_net, exit_signal, run_signal, detections
    weights = "runs/detect/yolov8n_rc_v13/weights/best.pt"
    img_size = 416
    conf_thres=0.4
    svo = None
    capture_thread = Thread(target=torch_thread, kwargs={
        'weights': weights, 
        'img_size': img_size, 
        "conf_thres": conf_thres
    })
    capture_thread.start()
    
    latencies = []

    print("Initializing Camera...")

    zed = sl.Camera()

    input_type = sl.InputType()
    if svo is not None:
        input_type.set_from_svo_file(svo)

    # Create a InitParameters object and set configuration parameters
    init_params = sl.InitParameters(input_t=input_type, svo_real_time_mode=True)
    init_params.coordinate_units = sl.UNIT.METER
    init_params.depth_mode = sl.DEPTH_MODE.ULTRA  # QUALITY
    init_params.coordinate_system = sl.COORDINATE_SYSTEM.RIGHT_HANDED_Y_UP
    init_params.depth_maximum_distance = 50

    runtime_params = sl.RuntimeParameters()
    status = zed.open(init_params)

    if status != sl.ERROR_CODE.SUCCESS:
        print(repr(status))
        exit()

    image_left_tmp = sl.Mat()

    print("Initialized Camera")

    positional_tracking_parameters = sl.PositionalTrackingParameters()
    # If the camera is static, uncomment the following line to have better performances and boxes sticked to the ground.
    # positional_tracking_parameters.set_as_static = True
    zed.enable_positional_tracking(positional_tracking_parameters)

    obj_param = sl.ObjectDetectionParameters()
    obj_param.detection_model = sl.OBJECT_DETECTION_MODEL.CUSTOM_BOX_OBJECTS
    obj_param.enable_tracking = True
    zed.enable_object_detection(obj_param)

    objects = sl.Objects()
    obj_runtime_param = sl.ObjectDetectionRuntimeParameters()

    # Display
    camera_infos = zed.get_camera_information()
    camera_res = camera_infos.camera_configuration.resolution
    # Create OpenGL viewer
    # viewer = gl.GLViewer()
    point_cloud_res = sl.Resolution(min(camera_res.width, 720), min(camera_res.height, 404))
    point_cloud_render = sl.Mat()
    # viewer.init(camera_infos.camera_model, point_cloud_res, obj_param.enable_tracking)
    point_cloud = sl.Mat(point_cloud_res.width, point_cloud_res.height, sl.MAT_TYPE.F32_C4, sl.MEM.CPU)
    image_left = sl.Mat()
    # Utilities for 2D display
    display_resolution = sl.Resolution(min(camera_res.width, 1280), min(camera_res.height, 720))
    image_scale = [display_resolution.width / camera_res.width, display_resolution.height / camera_res.height]
    image_left_ocv = np.full((display_resolution.height, display_resolution.width, 4), [245, 239, 239, 255], np.uint8)

    # Utilities for tracks view
    camera_config = camera_infos.camera_configuration
    tracks_resolution = sl.Resolution(400, display_resolution.height)
    track_view_generator = cv_viewer.TrackingViewer(tracks_resolution, camera_config.fps, init_params.depth_maximum_distance)
    track_view_generator.set_camera_calibration(camera_config.calibration_parameters)
    image_track_ocv = np.zeros((tracks_resolution.height, tracks_resolution.width, 4), np.uint8)
    # Camera pose
    cam_w_pose = sl.Pose()

    # while viewer.is_available() and not exit_signal:
    while not exit_signal:
        if zed.grab(runtime_params) == sl.ERROR_CODE.SUCCESS:
            # -- Get the image
            
            lock.acquire()
            zed.retrieve_image(image_left_tmp, sl.VIEW.LEFT)
            
            image_net = image_left_tmp.get_data()
            lock.release()
            run_signal = True
            startbig = time.time()

            # -- Detection running on the other thread
            while run_signal:
                sleep(0.001)

            # Wait for detections
            lock.acquire()
            # -- Ingest detections
            zed.ingest_custom_box_objects(detections)
            lock.release()
            zed.retrieve_objects(objects, obj_runtime_param)

            # -- Display
            # Retrieve display data
            zed.retrieve_measure(point_cloud, sl.MEASURE.XYZRGBA, sl.MEM.CPU, point_cloud_res)
            point_cloud.copy_to(point_cloud_render)
            zed.retrieve_image(image_left, sl.VIEW.LEFT, sl.MEM.CPU, display_resolution)
            zed.get_position(cam_w_pose, sl.REFERENCE_FRAME.WORLD)

            # 3D rendering
            # viewer.updateData(point_cloud_render, objects)
            # 2D rendering
            np.copyto(image_left_ocv, image_left.get_data())
            cv_viewer.render_2D(image_left_ocv, image_scale, objects, obj_param.enable_tracking)
            global_image = cv2.hconcat([image_left_ocv, image_track_ocv])
            # Tracking view
            track_view_generator.generate_view(objects, cam_w_pose, image_track_ocv, objects.is_tracked)
            endbig = time.time()
            cv2.imshow("ZED | 2D View and Birds View", global_image)
            key = cv2.waitKey(10)
            if key == 27:
                exit_signal = True
                
            
            print(round(endbig-startbig, 2), "seconds")
            latencies.append(endbig-startbig)
        else:
            exit_signal = True

    # viewer.exit()
    exit_signal = True
    zed.close()
    
    print(round(avg(latencies), 3), "seconds per loop")




with torch.no_grad():
    main()
    
# 0.605 seconds per loop
# 0.605 seconds of latency


# in hallway, max distance = 4.3 meters
# 0.68 seconds per loop

# in outside, 5.6 max partly cloudy
# 80% outside partly cloudy


Intializing Network...Initializing Camera...
Initialized Camera




0: 256x416 (no detections), 1037.7ms
Speed: 7.8ms preprocess, 1037.7ms inference, 1.5ms postprocess per image at shape (1, 3, 256, 416)



4.51 seconds


0: 256x416 (no detections), 1015.7ms
Speed: 3.9ms preprocess, 1015.7ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 416)



1.06 seconds


...

0: 256x416 (no detections), 541.3ms
Speed: 3.0ms preprocess, 541.3ms inference, 0.9ms postprocess per image at shape (1, 3, 256, 416)



0.57 seconds


0: 256x416 (no detections), 609.7ms
Speed: 3.1ms preprocess, 609.7ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 416)


0.64 seconds
0.589 seconds per loop


: 

## For Detecting the Headlights
In [this paper from Innopolis University in Russia](https://github.com/TechToker/Vehicle-rear-lights-analyser/blob/master/FinalReport.pdf), researchers intended detect if the car in front of them were braking or not. 

The researchers use YOLOv4 to detect whether a car is present for each frame, and then cropp the frame to just the car.

To detect the presense of brakelights, they convert each image into YCrCb space, then removes all pixels that aren't pure red. They utilize the fact that the taillights are symmetrical in cars to weed away noise from other cars. They then apply a bounding box over the taillights.

To determine if the car is braking, it determines the brightness in the bounding boxes. It's important to note that it has to see what the car looks like when braking vs not braking to know which is brighter, and thus be able to distinguish when it's braking.

Our algorithm can utilize soemthing similar, but will have to identify the headlights a different way since headlights aren't red. Potentially, a ML model could be used to determine the headlights' position. Determining when its on vs off would be the same as the Innopolis paper.